In [1]:
from more_itertools import pairwise
import itertools
import igraph as ig
import random
import colorsys
from datetime import datetime, timedelta
from collections import Counter

In [2]:
period = 'april2022'
path = '../data/' + period + '/'

In [3]:
def to_stop_tuples(data_path):
    with open(data_path, 'rt') as f:
        # Remove empty line and new line tokens
        lines = filter(None, (line.rstrip() for line in f))
        # Skip first item, than pairwise
        for line1, line2 in pairwise(itertools.islice(lines, 1, None)):
            route_id1, _, _, departure_time1, stop_id1, stop_seq1, _ = line1.split(';')
            route_id2, _, arrival_time2, _, stop_id2, stop_seq2, _ = line2.split(';')
            if route_id1 == route_id2 and int(stop_seq2) > int(stop_seq1):
                if departure_time1 == '' or arrival_time2 == '':
                    yield (route_id1, stop_id1, stop_id2, 0)
                else:
                    departure_time1 = datetime.strptime(departure_time1,"%H:%M:%S").time()
                    departure_time1 = timedelta(hours=departure_time1.hour, minutes=departure_time1.minute, seconds=departure_time1.second)    
                    arrival_time2 = datetime.strptime(arrival_time2,"%H:%M:%S").time()
                    arrival_time2 = timedelta(hours=arrival_time2.hour, minutes=arrival_time2.minute, seconds=arrival_time2.second)         
                    yield (route_id1, stop_id1, stop_id2, (arrival_time2 - departure_time1).seconds)


In [4]:
def get_stops(data_path):
    stops = dict()
    with open(data_path, 'rt') as f:
        # Remove empty line and new line tokens
        lines = filter(None, (line.rstrip() for line in f))
        for l in itertools.islice(lines, 1, None):  # Skip first item
            stop_id, _, stop_name, _, stop_lat, stop_lon, _, _, _ = l.split(',')
            stops[stop_id] = (stop_name, float(stop_lat), float(stop_lon))
    return stops

In [5]:
def get_times_week(data_path):
    times_week = []
    with open(data_path, 'rt') as f:
        # Remove empty line and new line tokens
        lines = filter(None, (line.rstrip() for line in f))
        for l in itertools.islice(lines, 1, None):  # Skip first item
            _, service, trip, stop_id, _, times = l.split(';')
            times_week.append([service,trip,stop_id,int(times)])
    return times_week


In [6]:
def rand_color():
    h, s, l = random.random(), 0.5 + random.random()/2.0, 0.4 + random.random()/5.0
    r, g, b = [int(256*i) for i in colorsys.hls_to_rgb(h,l,s)]
    return (r / 255, g / 255, b / 255) 

In [7]:
tratte_data = path + 'processed/tratte.csv'
stops_data = path + 'original/stops.txt'
week_data = path + 'processed/weeks.csv'

In [8]:
# Tratte data
tratte_tuple = {(route_id, stop_id1, stop_id2) for route_id, stop_id1, stop_id2, _ in to_stop_tuples(tratte_data)}

# Stops data
stops = get_stops(stops_data)

# Routes data
routes = { route_id for route_id, _, _ in tratte_tuple }
# Routes color
routes_color = { route_id : rand_color() for route_id in routes }

In [9]:
# Get times a week
times_week = get_times_week(week_data)

In [10]:
from itertools import groupby
from statistics import mean

keyfunct = key=lambda v: (lambda route_id1, stop_id1, stop_id2, _: (route_id1, stop_id1, stop_id2))(*v)
grouped = groupby(sorted(to_stop_tuples(tratte_data), key=keyfunct), key=keyfunct)
tratta_time = { k : mean(tratta_time for _, _, _, tratta_time in g) for k, g in grouped }

In [12]:
g = ig.Graph(directed=True)

g.add_vertices(
    n = list(stops.keys()), 
    attributes = {
        'label': [label for label, _, _ in stops.values()],
        'lat': [lat for _, lat, _ in stops.values()],
        'lon': [lon for _, _, lon in stops.values()]
    }
)

g.add_edges(
    es = [(stop_id1, stop_id2) for _, stop_id1, stop_id2 in tratte_tuple],
    attributes = {
        'label': [route_id for route_id, _, _ in tratte_tuple],
        'color': [routes_color[route_id] for route_id, _, _ in tratte_tuple],
        'time_second': [tratta_time[route_id, stop_id1, stop_id2]  for route_id, stop_id1, stop_id2 in tratte_tuple]
    }
)

In [45]:
# adding num_train
g.es['num_train'] = 0
for i in range(len(times_week)-1):
    line1 = times_week[i]
    line2 = times_week[i+1]
    if line1[0] == line2[0] and line1[1] == line2[1]:
        source_v = g.vs.find(label = stops[line1[2]][0]).index
        target_v = g.vs.find(label = stops[line2[2]][0]).index
        edge = g.es.select(_source = source_v, _target = target_v)
        if len(edge) == 1:
            edge[0]['num_train'] = edge[0]['num_train'] + line1[3]
        elif len(edge) > 1:
            for i in range(len(edge)):
                edge[i]['num_train'] = edge[i]['num_train'] + line1[3]

for i in g.es:
    i['num_train'] = round(i['num_train'] / 7, 2)


In [56]:
# ATTENZIONE risultano più oggetti "arco" per la stessa coppia di nodi
sou = g.vs.find(label = 'STRADELLA').index
edge = g.es.select(_source = sou)
for i in edge:
    print(i.vertex_tuple)

(igraph.Vertex(<igraph.Graph object at 0x7f370b736220>, 291, {'name': '2790', 'label': 'STRADELLA', 'lat': 45.079286, 'lon': 9.300823}), igraph.Vertex(<igraph.Graph object at 0x7f370b736220>, 119, {'name': '746', 'label': 'BRONI', 'lat': 45.066163, 'lon': 9.258878}))
(igraph.Vertex(<igraph.Graph object at 0x7f370b736220>, 291, {'name': '2790', 'label': 'STRADELLA', 'lat': 45.079286, 'lon': 9.300823}), igraph.Vertex(<igraph.Graph object at 0x7f370b736220>, 63, {'name': '457', 'label': 'ARENA PO', 'lat': 45.08214, 'lon': 9.360389}))
(igraph.Vertex(<igraph.Graph object at 0x7f370b736220>, 291, {'name': '2790', 'label': 'STRADELLA', 'lat': 45.079286, 'lon': 9.300823}), igraph.Vertex(<igraph.Graph object at 0x7f370b736220>, 63, {'name': '457', 'label': 'ARENA PO', 'lat': 45.08214, 'lon': 9.360389}))
(igraph.Vertex(<igraph.Graph object at 0x7f370b736220>, 291, {'name': '2790', 'label': 'STRADELLA', 'lat': 45.079286, 'lon': 9.300823}), igraph.Vertex(<igraph.Graph object at 0x7f370b736220>, 11

In [28]:
g.summary()


'IGRAPH DN-- 428 1706 -- \n+ attr: label (v), lat (v), lon (v), name (v), color (e), label (e), num_train (e), time_second (e)'

In [41]:
# Save the graph
save_path = '../data/graphs/' + period + '.graphml' 
g.write_graphml(save_path)

In [43]:
visual_style = {} #Python dictionary
visual_style["edge_label"] = g.es['num_train']
visual_style["layout"] = g.layout_auto()

ig.plot(g, bbox=(0, 0, 3000, 3000), **visual_style)